# Analyze and Transform Fundamental Stock Data

In [178]:
import pandas as pd
from eod import EodHistoricalData
from functools import reduce
import plotly.express as px
from datetime import datetime, timedelta

# Importing and assigning the api key
with open("../eodHistoricalData-API.txt", "r") as f:
    api_key = f.read()
    
# EOD Historical Data client
client = EodHistoricalData(api_key)

In [79]:
# Getting a list of available tickers
tickers = client.get_exchange_symbols("US")

tickers = [i['Code'] for i in tickers if i['Type']=='Common Stock']

# Formatting Fundamental Data

In [324]:
def getFundamentals(ticker):
    
    # Getting data
    fund_data = client.get_fundamental_equity(ticker)
    
    # Financials
    bal = pd.DataFrame(fund_data['Financials']['Balance_Sheet']['quarterly']).T
    
    cf = pd.DataFrame(fund_data['Financials']['Cash_Flow']['quarterly']).T
    
    inc = pd.DataFrame(fund_data['Financials']['Income_Statement']['quarterly']).T
    
    # Earnings
    earn = pd.DataFrame(fund_data['Earnings']['History']).T
    
    # Merging them together
    df = reduce(
        lambda left,right: pd.merge(
            left,
            right,
            left_index=True, 
            right_index=True, 
            how='outer',
            suffixes=('', '_drop')
        ), 
        [bal, cf, inc, earn]
    )
    
    # Dropping redundant date and duplicate columns
    dup_cols = [i for i in df.columns if "date" in i or "Date" in i or "_drop" in i]
    
    df = df.drop(dup_cols, axis=1)
    
    return df

In [325]:
def getPrices(df, ticker):
    
    # Getting stock price at the time
    prices = client.get_prices_eod("MSFT", period='m')
    
    
    prices = pd.DataFrame(prices).set_index('date')[['adjusted_close', 'close', 'volume']]

    # Converting to date time
    prices.index = pd.to_datetime(prices.index)

    # Filling in missing price data
    prices = prices.reindex(
        pd.date_range(prices.index[0], prices.index[-1]),
        method='ffill'
    )
    
    # Converting back to string for merging later
    prices.index = prices.index.strftime("%Y-%m-%d")
        
    price_dates = [i for i in prices.index if i in df.index]
    
    prices = prices.loc[price_dates]

    # Joining together
    df = df.join(prices, how='outer')
    
    return df

In [326]:
def formatFundamentals(ticker, dropna=False):
    
    fund_data = getFundamentals(ticker)
    
    df = getPriceData(ticker, fund_data)
    
    if dropna:
        # Dropping mostly nan columns and rows if requested
        df = df.dropna(
            thresh=round(df.shape[1]*.5) # If 50% of the values in the row are Nans, drop the whole row
        ).dropna(
            axis=1,
            thresh=round(df.shape[0]*.5) # If 50% of the values in the columns are Nans, drop the whole column
        )
    
    return df

In [327]:
df = formatFundamentals("AAPL")

In [332]:
df

,currency_symbol,totalAssets,intangibleAssets,earningAssets,otherCurrentAssets,totalLiab,totalStockholderEquity,deferredLongTermLiab,otherCurrentLiab,commonStock,...,preferredStockAndOtherAdjustments,beforeAfterMarket,currency,epsActual,epsEstimate,epsDifference,surprisePercent,adjusted_close,close,volume
1985-09-30,None,936200000.0,None,None,None,385700000.0,550500000.0,None,None,None,...,None,NaN,NaN,NaN,NaN,NaN,NaN,0.0560,15.7472,36646398.0
1985-12-31,None,None,None,None,None,None,None,None,None,None,...,None,NaN,NaN,NaN,NaN,NaN,NaN,0.0783,21.9968,87247996.0
1986-03-31,None,None,None,None,None,None,None,None,None,None,...,None,NaN,NaN,NaN,NaN,NaN,NaN,0.1005,28.2464,187801592.0
1986-06-30,None,None,None,None,None,None,None,None,None,None,...,None,NaN,NaN,NaN,NaN,NaN,NaN,0.1276,35.8736,70761597.0
1986-09-30,None,1160100000.0,None,None,None,466000000.0,694100000.0,None,None,None,...,None,NaN,NaN,NaN,NaN,NaN,NaN,0.1192,33.4992,180790370.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-30,USD,329840000000.0,None,None,13641000000.0,265560000000.0,64280000000.0,None,43625000000.0,54989000000.0,...,None,AfterMarket,USD,1.3,1.0143,0.2857,28.1672,136.7551,136.9600,63261391.0
2021-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,None,USD,None,1.2400,None,None,141.5000,141.5000,88934200.0
2021-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,None,USD,None,None,None,None,NaN,NaN,NaN
2022-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,None,USD,None,None,None,None,NaN,NaN,NaN
